## 6.4 Bounded sequences

Using a static array, we can implement the sequence ADT for bounded
sequences,
because static arrays can't grow.
The capacity of the sequence is the length of the static array.
We must keep track of the sequence's current length, so that
we know which positions on the static array are occupied by sequence elements
and which ones are available for the sequence to grow.
We can't assume that the value `None` represents an available position,
because `None` is a potentially valid item of a sequence.

The next figure shows the data structures (a static array for the items and an integer for the size) for a sequence of
length three and capacity five, with values `True`, `None` and `0.2`.
The static array was initialised with five pointers to the same `None` object,
two of which have been replaced.
The size of the sequence is also the index of the next available position
because indices start at zero.
To illustrate this, I write the indices below the array.
As I wrote earlier, indices are also a form of reference.

<p id="fig-6.4.1"></p>

*[Figure 6.4.1](../33_Figures/Figures_06_4.ipynb#Figure-6.4.1)*

![Image 06_4_bounded.png](06_4_bounded.png)

Most sequence operations are straightforward to implement for bounded sequences,
by iterating over one or two static arrays (the input and output sequences).
The insert and remove methods require a bit more care.
I will implement insertion: the removal algorithm is largely symmetric.
Before that, let's take a little detour.

### 6.4.1 Outlining algorithms

In verbal and written communication with colleagues, algorithms are often
presented in a succinct way, just outlining the steps they carry out.
This is often sufficient to be able to implement the algorithm.
For example, here's how I'd outline the algorithm for the  `__str__` method
in the abstract `Sequence` class:

> The algorithm converts the sequence to a list,
> which is then converted to a string.
> To obtain the list, it starts by creating an empty list.
> It goes through the items in the sequence, appending each to the list.
> The list is converted to a string with the `str` function.

Algorithms are commands for the computer to execute, so if you prefer,
you can outline them in the imperative tense. Here's the same example:

> Convert the sequence to a list and then convert the list to a string.
> To obtain the list, first create an empty list,
> then go through the items in the sequence, appending each to the list.
> Convert the list to a string with the `str` function.

When outlining algorithms, you can assume the reader or listener already knows
the problem, i.e. what the inputs and output are and
what the algorithm is supposed to accomplish.
It's good practice to first give a 'bird's eye view' of the algorithm,
describing its various stages or phases, and then detail each one.
This helps the reader or listener see the wood for the trees.
An outline that sounds like reading the code aloud is not effective
communication: it's too detailed and it overwhelms the recipient because
they must keep many details in their head to follow the outline.

For subtle or complicated algorithms, the outline should help
the reader or listener understand not just what the algorithm does but also
why it works.
So do try to explain the rationale of any hairy part of an algorithm.

The outline has to be tailored to the knowledge level of the recipient.
The above outlines are Greek to most people,
or Latin if they happen to speak Greek.
In M269, the recipient is your tutor or a fellow student, so
you can assume they have the same knowledge as you.

Like everything else, outlining algorithms requires practice.
If you have a study buddy, call them, outline an algorithm in this book
and have them critique your outline, e.g. if it's missing important details
or if it's too detailed or confusing.
If your buddy can follow your outline just by listening to it,
without looking at the algorithm in their copy of the book,
then it's probably a good one. You can also post and critique outlines
in the M269 forums, e.g. of alternative solutions to the book's exercises.

With this in mind, let's tackle the insert operation.

### 6.4.2 Insertion

As usual, before designing an algorithm, we must think of problem instances.
For this operation, the edge cases are
lists with length zero and one, and inserting an item at the first, last and
after the last index. The latter corresponds to appending the item.
The table columns are the variables in the
operation definition: *values* is the input/output sequence;
*value* and *index* are the item to be inserted and its position.
The item to be inserted is not relevant, so I use the same for all test cases.
This saves me from writing a very repetitive column for *value*.
Remember that we write abstract sequences within parentheses.

Case | Pre-*values* | *index* | Post-*values*
-|-|-|-
length&nbsp;0  | ( ) | 0 | ('!')
length&nbsp;1, before  | ( 0 )  | 0  | ('!', 0)
length&nbsp;1, after  | ( 0 )  |  1 | (0, '!')
length&nbsp;2, before  | (0, 1)  | 0  | ('!', 0, 1)
length&nbsp;2, middle  | (0, 1)  | 1  | (0, '!', 1)
length&nbsp;2, after  | (0, 1)  | 2  | (0, 1, '!')

We can assume the bounded sequence has capacity for at least one more item.
The algorithm must somehow 'push' the items from the given index one position
to the right, to then put the item at the position given by the index.
Let's take one problem instance to help visualise the process.
I choose the fourth test case, inserting at the start of a length&nbsp;2&nbsp;sequence,
before handling the possibly trickier case of inserting in an empty sequence or
inserting at the end.

The sequence is (0, 1). To obtain the end result ('!', 0, 1) I must copy
each item to the next position up. I can't start by copying the first item, as
it would overwrite the value there and lose it forever: (0, 0).
I must start copying from the end:

- (0, 1, 1): copy the last item to the vacant position
- (0, 0, 1): copy the first item to the next position
- ('!', 0, 1): put the new item in the input position.

Here's a possible outline of the algorithm:

> Go through the items backwards, from the last one to
> the one at the given index, and copy each item to the next position.
> Put the given item at the given index.

From this, I can write an algorithm in English.
I can't iterate backwards over items, but I can iterate downwards over integers.

1. for each *position* from │*values*│ − 1 down to *index*:
   1. let *values*[*position* + 1] be *values*[*position*]
2. let *values*[*index*] be *value*

Step&nbsp;1.1 copies each item one position up, i.e. to the right.

Before moving on to implementing this algorithm in Python, let's check it works
for the other test cases. The penultimate case (inserting in the middle)
is similar to inserting at the start.
All other cases (first, third and last) append an item.
Does the algorithm work for them?

___

The item is appended when *index* = │*values*│. In that case, the loop
doesn't execute because the start position is larger than the end position.
The item is immediately put in its position.

The complexity of this algorithm is as stated in
[Section&nbsp;4.6.1](../04_Iteration/04_6_lists.ipynb#4.6.1-Modifying-sequences):
Θ(1) when appending and Θ(│*values*│ − *index*) otherwise.

### 6.4.3 The `BoundedSequence` class

Translating the insertion algorithm to code requires one more detail:
incrementing the size variable, because we're dealing with
concrete data structures and not abstract sequences.

Here's an implementation of the abstract class methods for bounded sequences.
In Python we write `class A(B):` to define *A* as a subclass of *B*.

In [1]:
%run -i ../m269_array
%run -i ../m269_sequence


class BoundedSequence(Sequence):
    """A sequence with a fixed capacity."""

    def __init__(self, capacity: int) -> None:
        """Create an empty sequence that can hold 'capacity' items.

        Preconditions: capacity >= 0
        """
        self.items = StaticArray(capacity)
        self.size = 0

    def capacity(self) -> int:
        """Return how many items the sequence can hold.

        Postconditions: the output is the value set at creation time
        """
        return self.items.length()

    def length(self) -> int:
        """Return the number of items in the sequence.

        Postconditions: 0 <= self.length() <= self.capacity()
        """
        return self.size

    def get_item(self, index: int) -> object:
        """Return the item at position index.

        Preconditions: 0 <= index < self.length()
        Postconditions: the output is the n-th item of self, with n = index + 1
        """
        return self.items.get_item(index)

    def set_item(self, index: int, item: object) -> None:
        """Replace the item at position index with the given one.

        Preconditions: 0 <= index < self.length()
        Postconditions: post-self.get_item(index) == item
        """
        self.items.set_item(index, item)

    def insert(self, index: int, item: object) -> None:
        """Insert item at position index.

        Preconditions: 0 <= index <= self.length() < self.capacity()
        Postconditions: post-self is the sequence
        pre-self.get_item(0), ..., pre-self.get_item(index - 1),
        item, pre-self.get_item(index), ...,
        pre-self.get_item(pre-self.length() - 1)
        """
        for position in range(self.size - 1, index, -1):
            self.items.set_item(position + 1, self.items.get_item(position))
        self.items.set_item(index, item)
        self.size = self.size + 1

The following example uses the `append` and `__str__` methods,
provided by the abstract superclass.

In [2]:
items = BoundedSequence(5)
print(items)
for item in ("ready", "set", "go!"):
    items.append(item)
    print(items)
print("Can have", items.capacity() - items.length(), "more items.")

[]
['ready']
['ready', 'set']
['ready', 'set', 'go!']
Can have 2 more items.


Let's run the tests created in the previous section,
for different capacities and lengths.
Each test takes an empty bounded sequence of some capacity
and grows it to a given length.

In [3]:
%run -i ../m269_test

for capacity in range(4):
    print("Testing capacity", capacity)
    test_init(BoundedSequence(capacity))
    for length in range(capacity + 1):
        print("Testing length", length)
        test_append(BoundedSequence(capacity), length)
        test_insert_start(BoundedSequence(capacity), length)
        test_set_item(BoundedSequence(capacity), length)

Testing capacity 0
Testing length 0
Testing capacity 1
Testing length 0
Testing length 1
Testing capacity 2
Testing length 0
Testing length 1
Testing length 2
insert at 0: n-th item FAILED for [0, None] : None instead of 1
Testing capacity 3
Testing length 0
Testing length 1
Testing length 2
insert at 0: n-th item FAILED for [0, None] : None instead of 1
Testing length 3
insert at 0: n-th item FAILED for [0, None, None] : None instead of 1
insert at 0: n-th item FAILED for [0, None, None] : None instead of 2


Oh dear! The test for inserting at the start failed for lists longer than one.
In all cases the resulting sequence is 0 followed by `None`, instead of being
0, 1, 2, ...

#### Exercise 6.4.1

Explain the error and how to fix it.
(By the way, this was a genuine error: I didn't do it on purpose.)

_Write your answer here._

[Hint](../31_Hints/Hints_06_4_01.ipynb)
[Answer](../32_Answers/Answers_06_4_01.ipynb)

#### Exercise 6.4.2

Fix the error in the `insert` method.
Rerun the tests.

[Answer](../32_Answers/Answers_06_4_02.ipynb)

#### Exercise 6.4.3 (optional)

Add a `remove` method to the `BoundedSequence` class above.
Run its code cell again, then run these tests:

In [4]:
for capacity in range(4):
    print("Testing capacity", capacity)
    for length in range(capacity + 1):
        print("Testing length", length)
        test_remove(BoundedSequence(capacity), length)

[Hint](../31_Hints/Hints_06_4_03.ipynb)

⟵ [Previous section](06_3_develop_type.ipynb) | [Up](06-introduction.ipynb) | [Next section](06_5_dynamic_array.ipynb) ⟶